In [ ]:
import pandas as pd
import geopandas as gpd
import janitor as jn
from criminologia_cdmx.etl import *
from criminologia_cdmx.modelos import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
datos = get_carpetas_desde_archivo("datos/descargas/carpetas_fiscalia.csv")
datos = agrega_ids_espaciales(datos)
fecha_inicio = "01-01-2019"
fecha_fin = "01-01-2022"
columna_y = 'delito'
valores_y = ['ROBO A CASA HABITACION SIN VIOLENCIA', 'ROBO A CASA HABITACION CON VIOLENCIA']
fecha_inicio = pd.to_datetime(fecha_inicio, dayfirst=True)
fecha_fin = pd.to_datetime(fecha_fin, dayfirst=True)
datos = datos.loc[datos['fecha_hechos'].between(fecha_inicio, fecha_fin)]
datos = datos.loc[datos[columna_y].isin(valores_y)]
st = (datos
      .groupby(['colonia_cve', 'delito'])
      .resample(rule='Y', on='fecha_hechos').size())


In [ ]:
st = st.reset_index()
st.rename({0:'conteo'}, axis=1, inplace=True)
st

,colonia_cve,delito,fecha_hechos,conteo
0,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2019-12-31,1
1,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2020-12-31,1
2,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2019-12-31,1
3,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2020-12-31,0
4,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2021-12-31,1
...,...,...,...,...
5057,1813.0,ROBO A CASA HABITACION CON VIOLENCIA,2020-12-31,1
5058,1813.0,ROBO A CASA HABITACION CON VIOLENCIA,2021-12-31,1
5059,1813.0,ROBO A CASA HABITACION SIN VIOLENCIA,2019-12-31,3
5060,1813.0,ROBO A CASA HABITACION SIN VIOLENCIA,2020-12-31,2


In [ ]:
fechas_todas = pd.date_range(fecha_inicio, fecha_fin, freq='Y')
delitos_todos = st.delito.unique()
st = (st.complete('colonia_cve', 
           {'fecha_hechos' : fechas_todas}, 
           {'delito' : delitos_todos}, 
           sort=True)
        .fillna(0))
st.head(20)

,colonia_cve,delito,fecha_hechos,conteo
0,1.0,ROBO A CASA HABITACION CON VIOLENCIA,2019-12-31,0.0
1,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2019-12-31,1.0
2,1.0,ROBO A CASA HABITACION CON VIOLENCIA,2020-12-31,0.0
3,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2020-12-31,1.0
4,1.0,ROBO A CASA HABITACION CON VIOLENCIA,2021-12-31,0.0
5,1.0,ROBO A CASA HABITACION SIN VIOLENCIA,2021-12-31,0.0
6,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2019-12-31,1.0
7,2.0,ROBO A CASA HABITACION SIN VIOLENCIA,2019-12-31,1.0
8,2.0,ROBO A CASA HABITACION CON VIOLENCIA,2020-12-31,0.0
9,2.0,ROBO A CASA HABITACION SIN VIOLENCIA,2020-12-31,1.0


In [ ]:
st.groupby('colonia_cve').size()

colonia_cve
1.0       6
2.0       6
3.0       6
4.0       6
5.0       6
         ..
1809.0    6
1810.0    6
1811.0    6
1813.0    6
1814.0    6
Length: 1556, dtype: int64